In [ ]:
import os
import pickle
import cv2
import dlib
import numpy as np
from mtcnn import MTCNN
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

In [ ]:
friends_folder = r"C:\Users\congnguyen\Documents\school\cpv301\user\database"

In [ ]:
detector = MTCNN()

In [ ]:
face_rec_model_path = r"C:\Users\congnguyen\Documents\school\cpv301\dlib_face_recognition_resnet_model_v1.dat"
shape_predictor_path = r"C:\Users\congnguyen\Documents\school\cpv301\shape_predictor_68_face_landmarks.dat"
sp = dlib.shape_predictor(shape_predictor_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

In [ ]:
embedding_face = {}

In [ ]:
for friend_name in os.listdir(friends_folder):
    friend_images = os.listdir(os.path.join(friends_folder, friend_name))
    for image_name in friend_images:
        image_path = os.path.join(friends_folder, friend_name, image_name)
        image = cv2.imread(image_path)
        faces = detector.detect_faces(image)

        for face in faces:
            x, y, w, h = face['box']
            face_img = image[y:y+h, x:x+w]
            shape = sp(image, dlib.rectangle(x, y, x+w, y+h))
            embedding_vector = np.array(facerec.compute_face_descriptor(image, shape))
            embedding_face[friend_name] = embedding_vector

In [ ]:
with open('face_embedding.pkl', 'wb') as file:
    pickle.dump(embedding_face, file)

In [ ]:
with open('face_embedding.pkl', 'rb') as file:
    embedding_face = pickle.load(file)

In [3]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)
    faces = detector.detect_faces(frame)
    names = defaultdict(float)

    for face in faces:
        x, y, w, h = face['box']
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        face_img = frame[y:y+h, x:x+w]
        shape = sp(frame, dlib.rectangle(x, y, x+w, y+h))
        embedding_vector = np.array(facerec.compute_face_descriptor(frame, shape))
        similarity = [cosine_similarity([embedding_vector], [embedding_f]) for _, embedding_f in embedding_face.items()]
        max_sim = max(similarity)
        index = similarity.index(max_sim)
        name = list(embedding_face.keys())[index]
        names[name] = max(names[name], max_sim)

    for name, sim in names.items():
        sim_value = sim[0][0]
        label = f"{name}:{sim_value:.2f}" if sim_value > 0.75 else 'Unknown'
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1)

    cv2.imshow('Face Recognition', frame)
    if cv2.waitKey(40) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
